In [16]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.express as px
from omegaconf.omegaconf import OmegaConf
import torch as t
from higgs import DATAROOT, COLNAMES, HiggsDataModule, HiggsClassifier
from higgs.main import *
from haikunator import Haikunator
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning import Trainer

In [9]:
# !pip install plotly
# !pip install omegaconf
# !pip install pytorch-lightning
!pip install haikunator

distutils: /opt/conda/include/python3.6m/UNKNOWN
sysconfig: /opt/conda/include/python3.6m
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.6m/UNKNOWN
sysconfig: /opt/conda/include/python3.6m
user = False
home = None
root = None
prefix = None


In [14]:
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 30

np.set_printoptions(
    edgeitems=30, 
    linewidth=100000, 
    precision=3,
    suppress=True
)

In [15]:
hyperparams = """
n_epochs: 5
batch_size: 5
lr: 0.01
true_cutoff: 0.5
"""
hparams = OmegaConf.create(hyperparams)

In [4]:
higgs = pd.read_csv(DATAROOT/"HIGGS.csv", header=None, names=COLNAMES)

In [5]:
higgs.head()

,label,lepton_pt,lepton_eta,lepton_phi,missing_energy_magnitude,missing_energy_phi,jet_1_pt,jet_1_eta,jet_1_phi,jet_1_b-tag,jet_2_pt,jet_2_eta,jet_2_phi,jet_2_b-tag,jet_3_pt,jet_3_eta,jet_3_phi,jet_3_b-tag,jet_4_pt,jet_4_eta,jet_4_phi,jet_4_b-tag,m_jj,m_jjj,m_lv,m_jlv,m_bb,m_wbb,m_wwbb
0,1.0,0.869293,-0.635082,0.225690,0.327470,-0.689993,0.754202,-0.248573,-1.092064,0.000000,1.374992,-0.653674,0.930349,1.107436,1.138904,-1.578198,-1.046985,0.000000,0.657930,-0.010455,-0.045767,3.101961,1.353760,0.979563,0.978076,0.920005,0.721657,0.988751,0.876678
1,1.0,0.907542,0.329147,0.359412,1.497970,-0.313010,1.095531,-0.557525,-1.588230,2.173076,0.812581,-0.213642,1.271015,2.214872,0.499994,-1.261432,0.732156,0.000000,0.398701,-1.138930,-0.000819,0.000000,0.302220,0.833048,0.985700,0.978098,0.779732,0.992356,0.798343
2,1.0,0.798835,1.470639,-1.635975,0.453773,0.425629,1.104875,1.282322,1.381664,0.000000,0.851737,1.540659,-0.819690,2.214872,0.993490,0.356080,-0.208778,2.548224,1.256955,1.128848,0.900461,0.000000,0.909753,1.108330,0.985692,0.951331,0.803252,0.865924,0.780118
3,0.0,1.344385,-0.876626,0.935913,1.992050,0.882454,1.786066,-1.646778,-0.942383,0.000000,2.423265,-0.676016,0.736159,2.214872,1.298720,-1.430738,-0.364658,0.000000,0.745313,-0.678379,-1.360356,0.000000,0.946652,1.028704,0.998656,0.728281,0.869200,1.026736,0.957904
4,1.0,1.105009,0.321356,1.522401,0.882808,-1.205349,0.681466,-1.070464,-0.921871,0.000000,0.800872,1.020974,0.971407,2.214872,0.596761,-0.350273,0.631194,0.000000,0.479999,-0.373566,0.113041,0.000000,0.755856,1.361057,0.986610,0.838085,1.133295,0.872245,0.808487


In [8]:
shuffled = higgs.sample(frac=1.0)

In [9]:
shuffled.head()

,label,lepton_pt,lepton_eta,lepton_phi,missing_energy_magnitude,missing_energy_phi,jet_1_pt,jet_1_eta,jet_1_phi,jet_1_b-tag,jet_2_pt,jet_2_eta,jet_2_phi,jet_2_b-tag,jet_3_pt,jet_3_eta,jet_3_phi,jet_3_b-tag,jet_4_pt,jet_4_eta,jet_4_phi,jet_4_b-tag,m_jj,m_jjj,m_lv,m_jlv,m_bb,m_wbb,m_wwbb
9634744,0.0,3.415133,-1.146415,0.250659,0.863785,1.610422,1.555765,-0.921929,-1.466267,2.173076,0.976507,-0.458428,1.290988,0.000000,0.830338,-0.938294,-0.724129,1.274112,0.564218,1.516110,-0.462506,0.000000,1.674574,1.288693,2.239219,2.092927,0.881589,1.355154,1.391124
8947576,1.0,0.701291,-0.084789,-0.433033,0.765864,0.939437,0.683665,0.579259,-0.722296,2.173076,0.799110,1.031659,0.222386,2.214872,0.813654,0.162197,-1.495212,0.000000,1.217523,0.296024,0.798357,0.000000,0.927360,0.944335,0.988194,0.835206,0.859071,0.717539,0.710942
3807330,1.0,1.044616,-0.153941,-0.270458,0.944477,-0.295409,1.091134,-0.741708,1.616719,2.173076,0.588095,-0.763439,-0.421320,0.000000,0.883902,-1.384315,0.949612,2.548224,0.588072,0.375143,1.664575,0.000000,0.887106,0.986759,0.994645,0.900189,0.845608,0.945084,0.808487
7500533,1.0,1.411366,-0.245494,-1.384623,0.595813,-0.259397,1.126311,0.044535,-1.721280,2.173076,2.015967,-0.516711,-0.165542,0.000000,1.151725,0.680129,0.514700,0.000000,1.856710,0.106974,-1.031848,3.101961,0.966844,1.167639,1.078978,0.862449,0.894523,1.073868,0.941294
5240197,1.0,0.610518,-0.473403,0.098627,1.394401,-1.078242,0.900865,0.432705,-0.554874,0.000000,1.142826,-0.595392,1.083482,2.214872,0.688963,0.669206,1.349576,0.000000,0.802270,0.654971,0.705687,3.101961,0.871008,0.834473,1.114825,1.039286,0.715881,0.866846,0.804087


In [10]:
higgs.iloc[9634744]

label                       0.000000
lepton_pt                   3.415133
lepton_eta                 -1.146415
lepton_phi                  0.250659
missing_energy_magnitude    0.863785
missing_energy_phi          1.610422
jet_1_pt                    1.555765
jet_1_eta                  -0.921929
jet_1_phi                  -1.466267
jet_1_b-tag                 2.173076
jet_2_pt                    0.976507
jet_2_eta                  -0.458428
jet_2_phi                   1.290988
jet_2_b-tag                 0.000000
jet_3_pt                    0.830338
jet_3_eta                  -0.938294
jet_3_phi                  -0.724129
jet_3_b-tag                 1.274112
jet_4_pt                    0.564218
jet_4_eta                   1.516110
jet_4_phi                  -0.462506
jet_4_b-tag                 0.000000
m_jj                        1.674574
m_jjj                       1.288693
m_lv                        2.239219
m_jlv                       2.092927
m_bb                        0.881589
m